**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ sklearn 0.20.2

In [13]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [1]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [2]:
iris = datasets.load_iris()

In [3]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [4]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [5]:
# смотрим на параметры которые мы можем подобрать 
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [7]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}
# если мы построим декартово произведение на этих параметрах, то мы получим точки со всеми возможными наборами параметров

In [8]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)
# выбрали "стратегию" кросс-валидации

### Подбор параметров и оценка качества

#### Grid search

Работаем с не очень большим датасетом и\или перебираем не очень большое число параметров.

In [11]:
# классификатор сетка метрика_которую_будем_проверять стратегия_кроссвалидации
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [14]:
%%time
# обучение сетки, перебираем все варианты из декартового произведения (200 комбинаций + кросс валидация)
grid_cv.fit(train_data, train_labels)

CPU times: user 4.73 s, sys: 37.2 ms, total: 4.77 s
Wall time: 4.8 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2...
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge

In [15]:
# самый лучший классификатор
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=9,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=0, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [35]:
# выведем лучшее оценка и соответсвутющие наборы параметров
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.000325, 'loss': 'squared_hinge', 'max_iter': 9, 'penalty': 'l1'}


In [18]:
# посмотрим оценки на всех возможных наборах
grid_cv.cv_results_

{'mean_fit_time': array([0.00234916, 0.00174322, 0.0017554 , 0.00182278, 0.00180655,
        0.00175333, 0.0017256 , 0.00169973, 0.00165136, 0.00160289,
        0.00190566, 0.00168984, 0.00179551, 0.00173998, 0.00179636,
        0.00179062, 0.00181026, 0.00176084, 0.00187628, 0.0019609 ,
        0.00188148, 0.00175974, 0.00201275, 0.00193768, 0.00184536,
        0.00211284, 0.00203664, 0.00183501, 0.00184708, 0.00182993,
        0.00180652, 0.00195217, 0.00179374, 0.00179749, 0.00228617,
        0.00180595, 0.00195737, 0.0017493 , 0.00191572, 0.00182164,
        0.00186491, 0.00167024, 0.00179029, 0.00187066, 0.00178707,
        0.00182676, 0.00188155, 0.00201082, 0.00180922, 0.00194571,
        0.00202839, 0.00188158, 0.0019501 , 0.00187752, 0.00189404,
        0.00202575, 0.00192857, 0.00195553, 0.00208266, 0.00203466,
        0.00193815, 0.00214028, 0.00193415, 0.00181808, 0.00204177,
        0.00197515, 0.00185821, 0.0016892 , 0.00191357, 0.00169826,
        0.00164797, 0.00162327,

#### Randomized grid search
(случайный поиск по сетке)

работаем с больмшим набором данных, перебираем большое количество параметров

In [20]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [21]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 501 ms, sys: 15.4 ms, total: 516 ms
Wall time: 523 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   error_score=nan,
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n_jobs=None,
                                           penal...
                                           verbose=0, warm_start=False),
                   iid='deprecated', n_iter=20, n_jobs=None,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.00

In [22]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}
